In [33]:
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

# 15 Predefined Scenarios
SCENARIOS = {
    "Economic Recession": {"GDP Growth": -5, "Inflation": 2, "Loan Default Rate": 10, "Liquidity Drop": 15},
    "High Inflation": {"GDP Growth": 0, "Inflation": 8, "Loan Default Rate": 5, "Liquidity Drop": 10},
    "Market Crash": {"GDP Growth": -3, "Inflation": 1, "Loan Default Rate": 12, "Liquidity Drop": 20},
    "Credit Crisis": {"GDP Growth": -7, "Inflation": 3, "Loan Default Rate": 15, "Liquidity Drop": 25},
    "Interest Rate Hike": {"GDP Growth": -2, "Inflation": 6, "Loan Default Rate": 8, "Liquidity Drop": 10},
    "Pandemic Outbreak": {"GDP Growth": -8, "Inflation": 2, "Loan Default Rate": 20, "Liquidity Drop": 30},
    "Geo-Political Conflict": {"GDP Growth": -6, "Inflation": 5, "Loan Default Rate": 10, "Liquidity Drop": 20},
    "Technology Bubble Burst": {"GDP Growth": -4, "Inflation": 1, "Loan Default Rate": 12, "Liquidity Drop": 18},
    "Climate Change Impact": {"GDP Growth": -3, "Inflation": 4, "Loan Default Rate": 6, "Liquidity Drop": 10},
    "Cybersecurity Breach": {"GDP Growth": -2, "Inflation": 1, "Loan Default Rate": 8, "Liquidity Drop": 15},
    "Regulatory Tightening": {"GDP Growth": -1, "Inflation": 3, "Loan Default Rate": 5, "Liquidity Drop": 12},
    "Commodity Price Shock": {"GDP Growth": -5, "Inflation": 7, "Loan Default Rate": 10, "Liquidity Drop": 15},
    "Natural Disaster": {"GDP Growth": -6, "Inflation": 3, "Loan Default Rate": 12, "Liquidity Drop": 20},
    "Energy Crisis": {"GDP Growth": -4, "Inflation": 6, "Loan Default Rate": 10, "Liquidity Drop": 18},
    "Bank Run": {"GDP Growth": -8, "Inflation": 2, "Loan Default Rate": 25, "Liquidity Drop": 40},
}
#Function to calculate Capital Adequacy Ratio (CAR)
def calculate_car(capital, rwa):
    return (capital / rwa) * 100

#Stress Testing Function
def perform_stress_test(scenario, capital, rwa, liquidity):
    stress = SCENARIOS[scenario]
    gdp_impact = stress["GDP Growth"] * 0.01 * rwa
    loan_defaults = stress["Loan Default Rate"] * 0.01 * rwa
    liquidity_drop = stress["Liquidity Drop"] * 0.01 * liquidity
    post_stress_rwa = rwa + loan_defaults
    post_stress_capital = capital - loan_defaults - gdp_impact
    post_stress_liquidity = liquidity - liquidity_drop
    car = calculate_car(post_stress_capital, post_stress_rwa)
    return car, post_stress_capital, post_stress_liquidity

In [35]:
#stream lit
st.title("Capital Adequacy Stress Testing Dashboard")
#User Inputs
st.sidebar.header("Regulatory Inputs")
min_car = st.sidebar.number_input("Minimum CAR (%)", value = 8.0)
initital_capital = st.sidebar.number_input("Initial Capital ($M)", value= 1000.0)
initial_rwa = st.sidebar.number_input("Initial Risk-Weighted Assets (RWA) ($M)", value = 5000.0)
initial_liquidity = st.sidebar.number_input("Initial Liquidity ($M)", value = 2000.0)

st.sidebar.header("Scenario Selection")
selected_scenario = st.sidebar.selectbox("Select Stress Testing Scenario", list(SCENARIOS.keys()))

# Modify Scenario Parameters
st.sidebar.header("Adjust Scenario Parameters")
gdp_growth = st.sidebar.slider("GDP Growth (%)", -10, 5, SCENARIOS[selected_scenario]["GDP Growth"])
inflation = st.sidebar.slider("Inflation Rate (%)", 0, 10, SCENARIOS[selected_scenario]["Inflation"])
loan_default_rate = st.sidebar.slider("Loan Default Rate (%)", 0, 30, SCENARIOS[selected_scenario]["Loan Default Rate"])
liquidity_drop = st.sidebar.slider("Liquidity Drop (%)", 0, 50, SCENARIOS[selected_scenario]["Liquidity Drop"])

# Update scenario parameters based on user inputs
SCENARIOS[selected_scenario] = {
    "GDP Growth": gdp_growth,
    "Inflation": inflation,
    "Loan Default Rate": loan_default_rate,
    "Liquidity Drop": liquidity_drop,
}

# Perform Stress Test
car, post_capital, post_liquidity = perform_stress_test(
    selected_scenario, initial_capital, initial_rwa, initial_liquidity
)

# Display Results
st.subheader(f"Results for {selected_scenario}")
st.write(f"**Post-Stress Capital Adequacy Ratio (CAR):** {car:.2f}%")
st.write(f"**Post-Stress Capital:** ${post_capital:.2f}M")
st.write(f"**Post-Stress Liquidity:** ${post_liquidity:.2f}M")

if car < min_car:
    st.error("Warning: CAR is below the regulatory minimum!")
else:
    st.success("CAR meets the regulatory requirements.")

# Visualization
st.subheader("Capital Adequacy Over Scenarios")
scenario_names = list(SCENARIOS.keys())
car_values = [
    perform_stress_test(scenario, initial_capital, initial_rwa, initial_liquidity)[0]
    for scenario in scenario_names
]

plt.figure(figsize=(12, 6))
plt.bar(scenario_names, car_values, color="skyblue")
plt.axhline(min_car, color="red", linestyle="--", label="Regulatory Minimum")
plt.xticks(rotation=45, fontsize=10)
plt.ylabel("CAR (%)")
plt.title("Capital Adequacy Ratio Across Scenarios")
plt.legend()
st.pyplot(plt)

DeltaGenerator()